In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import sys
#change to any directory you have to store the repo
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling')

from src.features.metric import diversity, get_topic_coherence
from models.atten_model import MODEL_ATT_COMP
import swifter
from src.models.utils import sinkhorn_torch
from sklearn.preprocessing import MultiLabelBinarizer


In [2]:
import pandas as pd
from collections import Counter
from stop_words import get_stop_words
import swifter
from scipy.sparse import csr_matrix, save_npz
import numpy as np
import gc
import torch
stop_words = get_stop_words('en')
import re
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk.corpus import wordnet
import torch
from torch.nn.functional import normalize
import pickle 
import torch.nn.functional as F 
import numpy as np 
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.nn import init
from models.NVDM import sinkhorn_torch, negative_sampling_prior, optimal_transport_prior,  NormalParameter, get_mlp, EmbTopic, NSSTM, OTETM
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words, generate_bow
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score

In [ ]:
#data import
news = pd.read_csv('https://raw.githubusercontent.com/yumeng5/WeSTClass/master/agnews/dataset.csv', 
                   error_bad_lines=False,
                   names = ['class', 'title', 'description'])
news['text'] = news.swifter.apply(lambda x: ' '.join(remove_stopWords(x['title'] + x['description'])), axis=1)
news['clean_text']  = news.apply(lambda x: x['text'].split(), axis = 1)
#get clean data
common_words_ct = Counter([j for i in news['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 100)
word_track = {i: ind for ind, i in enumerate(common_words)}
index_track = {ind: i for ind, i in enumerate(common_words)}
news['index_num'] = news.swifter.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

In [ ]:
#change it to any location you save your embeddings
vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/agnews/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = news, common_words = common_words)


In [ ]:
seed_topic_list  = [['government', 'military', 'war'],
 ['basketball', 'football', 'athlete'],
 ['stock', 'market', 'industry'],
 ['computer', 'telescope', 'software']]
labels = [[word_track[j] for j in i] for i in seed_topic_list ]

In [ ]:
numb_embeddings = len(seed_topic_list) + 1
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1], 512, 64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
embedding.weight.requires_grad=False
topics = EmbTopic(embedding = embedding,
                  k = numb_embeddings)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NSSTM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics,
            #prior_penalty = 1, 
            diversity_penalty = 0, 
            iter1 = 15,
            iter2 = 20,
            beta = 0.25,
            gamma = 0.25,
            index = labels
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)


scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=40)

epochs = 25
for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)
